# Teste Unitário - Aquisição

**Objetivo**: Gerar as bases amostrais do pacote de aquisição

---

## Cabeçalho 

### Imports 

In [ ]:
import gc
import os
import pyunpack
import rarfile
import shutil
import sys
import zipfile
import numpy as np
import pandas as pd

from io import BytesIO
from pathlib import Path
from tqdm import tqdm

### Configurações 

In [ ]:
%config Completer.use_jedi = False

### Caminhos 

In [ ]:
try:
    PASTA_NOTEBOOK / ""
except:
    PASTA_NOTEBOOK = Path(os.getcwd())
    
PASTA_PROJETO = PASTA_NOTEBOOK.parent
PASTA_DADOS = PASTA_PROJETO / "dados"
PASTA_COMPLETO = PASTA_DADOS / "completo"
PASTA_EXTERNO = PASTA_COMPLETO / "externo"
PASTA_TESTE = PASTA_PROJETO / "src/tests"
PASTA_TESTE_DADOS = PASTA_TESTE / "dados"
PASTA_TESTE_SAIDA = PASTA_TESTE / "saida"
PASTA_TESTE_CODIGO = PASTA_TESTE / "codigos"
PASTA_TESTE_AQUIS = PASTA_TESTE / "aquisicao"

os.chdir(PASTA_PROJETO)

---

## Códigos 

In [ ]:
docentes = pd.read_pickle(PASTA_TESTE_CODIGO / "docentes.pkl").values
escolas = pd.read_pickle(PASTA_TESTE_CODIGO / "escolas.pkl").values
gestores = pd.read_pickle(PASTA_TESTE_CODIGO / "gestores.pkl").values
matriculas = pd.read_pickle(PASTA_TESTE_CODIGO / "matriculas.pkl").values
turmas = pd.read_pickle(PASTA_TESTE_CODIGO / "turmas.pkl").values

---

## Censo Escolar 

In [ ]:
caminho_entrada = PASTA_EXTERNO / "censo_escolar"
caminho_saida = PASTA_TESTE_DADOS / "externo/censo_escolar"
caminho_saida.mkdir(exist_ok=True, parents=True)
tabelas = {
    "docentes_co": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_nordeste": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_norte": [docentes, ("ID_DOCETE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_sudeste": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_sul": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "escolas": [escolas, ("PK_COD_ENTIDADE", "CO_ENTIDADE")], 
    "gestor": [gestores, ("ID_GESTOR",)], 
    "matricula_co": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_nordeste": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_norte": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_sudeste": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_sul": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "turmas": [turmas, ("ID_TURMA", "PK_COD_TURMA")], 
}

# para cada arquivo do censo demográfico
for censo in tqdm(os.listdir(caminho_entrada)):
    # abre o arquivo zip com o conteúdo do censo
    with zipfile.ZipFile(caminho_entrada / f"{censo}") as zin:
        # lista os arquivos que precisam de uma versão amostral
        arqs = [
            f 
            for tabela in tabelas
            for f in zin.namelist() 
            if f"{tabela}." in f.lower()
        ]
        
        if len(arqs) == 0:
            continue
        
        # cria uma cópia do arquivo zip na pasta de destino
        with zipfile.ZipFile(
            caminho_saida / f"{censo}", 
            mode="w", 
            compression=zipfile.ZIP_DEFLATED, 
            compresslevel=9
        ) as zout:
            # para cada item no arquivo de entrada
            # for item in zin.infolist():
            #     # verifica se o item faz parte de um dos arquivos amostrais
            #     if item.filename not in arqs:
            #         # se não, o copia para o novo zip
            #         zout.writestr(item, zin.read(item.filename))
            
            # para os demais itens
            for arq in arqs:
                tabela = arq.lower().split("/")[-1].split(".")[0]
                arq_zip = arq.split("/")[-1].split(".")[0]
                
                # carrega o arquivo CSV
                if ".csv" in arq.lower():
                    df = pd.read_csv(zin.open(arq), encoding="latin-1", sep="|")
                elif ".zip" in arq.lower():
                    with zipfile.ZipFile(zin.open(arq)) as z2:
                        para_abrir = z2.namelist()[0]
                        df = pd.read_csv(z2.open(para_abrir), encoding="latin-1", sep="|")
                elif ".rar" in arq.lower():
                    zin.extract(arq, path=caminho_entrada)
                    pyunpack.Archive(caminho_entrada / f"{arq}").extractall(caminho_entrada)
                    csv = [
                        f
                        for f in os.listdir(caminho_entrada)
                        if f"{tabela}." in f.lower()
                    ][0]
                    df = pd.read_csv(caminho_entrada / f"{csv}", encoding="latin-1", sep="|")
                    shutil.rmtree(caminho_entrada / f"{arq.split('/')[0]}")
                    os.remove(caminho_entrada / f"{csv}")

                # aplica os filtros sobre o arquivo
                col = [c for c in tabelas[tabela][1] if c in df][0]
                df = df.loc[lambda f: f[col].isin(tabelas[tabela][0])]
                
                # exporta o resultado
                # df.to_parquet(PASTA_TESTE_AQUIS / f"intermediario/{tabela}.parquet", index=False)
                if ".csv" in arq.lower():
                    txt = df.to_csv(encoding="latin-1", sep="|", index=False)
                    zout.writestr(arq, txt)
                elif ".zip" in arq.lower():
                    with zipfile.ZipFile(
                        caminho_saida / f"{arq_zip}.zip", 
                        mode="w", 
                        compression=zipfile.ZIP_DEFLATED, 
                        compresslevel=9
                    ) as zinner:
                        txt = df.to_csv(encoding="latin-1", sep="|", index=False)
                        zinner.writestr(f"{arq_zip}.csv", txt)
                    zout.write(f"{caminho_saida / arq_zip}.zip", arq)
                    os.remove(f"{caminho_saida / arq_zip}.zip")
                elif ".rar" in arq.lower():
                    os.chdir(caminho_saida)
                    df.to_csv(f"{arq_zip}.csv", encoding="latin-1", sep="|", index=False)
                    if "win" in sys.platform:
                        os.system(f"{PASTA_PROJETO / 'suporte/WinRAR/rar'} a {arq_zip}.rar {arq_zip}.csv")
                    else:
                        os.system(f"rar a {arq_zip}.rar {arq_zip}.csv")
                    zout.write(f"{caminho_saida / arq_zip}.rar", arq)
                    os.remove(f"{caminho_saida / arq_zip}.csv")
                    os.remove(f"{caminho_saida / arq_zip}.rar")
                    os.chdir(PASTA_PROJETO)

---

## IDEB 

In [ ]:
caminho_entrada = PASTA_EXTERNO / "ideb"
caminho_saida = PASTA_TESTE_DADOS / "externo/ideb"
caminho_saida.mkdir(exist_ok=True, parents=True)

for f in tqdm(os.listdir(caminho_entrada)):
    nome = os.path.splitext(f)[0]
    with zipfile.ZipFile(caminho_entrada / f) as z:
        data = pd.read_excel(z.open(f"{nome}/{nome}.xlsx"), header=None)
        sample = (
            data.head(10)
            .append(data[data[3].astype("str").isin(escolas.astype("str"))])
            .append(data.tail(3))
        )
        
    with zipfile.ZipFile(
        caminho_saida / f, 
        mode="w", 
        compression=zipfile.ZIP_DEFLATED, 
        compresslevel=9
    ) as zout:
        sample.to_excel(caminho_saida / f"{nome}.xlsx", header=None, index=False)
        zout.write(caminho_saida / f"{nome}.xlsx", f"{nome}/{nome}.xlsx")
        os.remove(caminho_saida / f"{nome}.xlsx")

---